In [1]:
import numpy as np

K = 0.01720209908 #U.A.^(3/2)/d
a = 1.0 # U.A.
e = 0.016674
Mr = 142.28337 #° para el 28 de mayo de 2014

# Ω ω i
i = 0.0
Ω = 0.0
ω = 102.9777

## 1) Buscamos hallar M

In [2]:
n = K*180/np.pi * (np.sqrt(1 + 1/3497.9018))/a**(3/2)
print("El movimiento medio es n = ", n, "°/d")

El movimiento medio es n =  0.9857485515374813 °/d


In [3]:
from datetime import datetime

def days_between(utc_time1, utc_time2): #Nos retorna la diferencia en días con fracción decimal
  
  time1 = datetime.strptime(utc_time1, "%Y-%m-%d %H:%M:%S")
  time2 = datetime.strptime(utc_time2, "%Y-%m-%d %H:%M:%S")
  delta = time2 - time1
  return delta.total_seconds() / (24 * 3600) #how many days with decimals

# Aplicación:
utc_time1 = "2014-05-28 00:00:00"
utc_time2 = "2014-06-21 00:00:00"

days_diff = days_between(utc_time1, utc_time2)
print(f"Hay {days_diff:.4f} días entre {utc_time1} y {utc_time2}.")

Hay 24.0000 días entre 2014-05-28 00:00:00 y 2014-06-21 00:00:00.


In [4]:
M = n*days_diff + Mr
print("La anomalía media es M = ", M,  "°")

La anomalía media es M =  165.94133523689953 °


## 2) Con la ecuación de Kepler hallamos la anomalía eccéntrica

In [5]:
from scipy.optimize import fsolve
import math
# Ecuación de Kepler en grados:
f = lambda E : M + e*(180/np.pi)*np.sin(np.radians(E)) - E

roots = fsolve(f, [100, 3002])
E = roots[0]

print("La anomalía eccéntrica es E = ", E, "°")


La anomalía eccéntrica es E =  166.16970860877055 °


## 3) Hallamos $\theta$ y $r$ usando:

In [6]:
r = a*(1 - e*np.cos(np.radians(E)))

print("El radio vector es r =", r, "U.A.")

El radio vector es r = 1.0161905879797808 U.A.


In [7]:
θ = 2*np.arctan(np.sqrt((1+e)/(1-e))*np.tan(np.radians(E/2)))
θ = np.degrees(θ)
print("La anomalía verdadera es θ = ", θ, "°")

La anomalía verdadera es θ =  166.39626339642263 °


In [8]:
#CONTROL:
r = a*(1 - e**2)/(1 + e*np.cos(np.radians(θ)))
print("El radio vector mediante la otra forma es r =", r, "U.A.")

El radio vector mediante la otra forma es r = 1.0161905879797808 U.A.


## 4) Posición con respecto al Sol en el plano fundamental (X, Y, Z)

In [9]:
x= r*( np.cos(np.radians(Ω)) * np.cos(np.radians(ω+θ)) - np.sin(np.radians(Ω)) * np.sin(np.radians(ω+θ)) * np.cos(np.radians(i)) )
y= r*( np.sin(np.radians(Ω)) * np.cos(np.radians(ω+θ)) + np.cos(np.radians(Ω)) * np.sin(np.radians(ω+θ)) * np.cos(np.radians(i)) )
z = r * np.sin(np.radians(ω+θ)) * np.sin(np.radians(i))

print("x =",  x, "U.A.")
print("y =", y, "U.A.")
print("z =", z, "U.A.")

x = -0.011103083881512035 U.A.
y = -1.0161299290085952 U.A.
z = -0.0 U.A.


## 5) Hallamos las geocéntricas del SOL

In [10]:
ξ = -x
η = -y
ζ = -z

print("Las coordenadas geocéntricas eclípticas del SOL son:")

print("ξ = ", ξ, "U.A.")
print("η = ", η, "U.A.")
print("ζ = ", ζ, "U.A.")

Las coordenadas geocéntricas eclípticas del SOL son:
ξ =  0.011103083881512035 U.A.
η =  1.0161299290085952 U.A.
ζ =  0.0 U.A.


In [11]:
from useful_functions import DECtoHMS
from useful_functions import DECtoDMS
from useful_functions import DMStoDEC

ε_2000 = DMStoDEC(23, 26, 21.406)



Rminus_ε= np.array([[1, 0, 0],
                       [0, math.cos(math.radians(ε_2000 )), -math.sin(math.radians(ε_2000 ))], 
                       [0, math.sin(math.radians(ε_2000 )), math.cos(math.radians(ε_2000 ))]])

r_geo_eclip = np.array([ξ, η, ζ ])

r_geo_ecua = Rminus_ε @ r_geo_eclip

ξp = r_geo_ecua[0]
ηp = r_geo_ecua[1]
ζp = r_geo_ecua[2]


print("Las coordenadas geocéntricas ecuatoriales del SOL son:")

print("ξ' = ", ξp, "U.A.")
print("η' = ", ηp, "U.A.")
print("ζ' = ", ζp, "U.A.")

Las coordenadas geocéntricas ecuatoriales del SOL son:
ξ' =  0.011103083881512035 U.A.
η' =  0.932281064899538 U.A.
ζ' =  0.4041930833856465 U.A.


## 7) Pasamos a coordenadas ecuatoriales absolutas

In [12]:
ρ = np.linalg.norm(r_geo_ecua)
α = np.degrees(np.arctan2(ηp, ξp))
δ = np.degrees(np.arcsin(ζp/ρ))

print("Las coordenadas ecuatoriales absolutas en DEG:")
print("ρ = ", ρ, "U.A.")
print("α = ", α , "°")
print("δ = ", δ, "°")

Las coordenadas ecuatoriales absolutas en DEG:
ρ =  1.0161905879797808 U.A.
α =  89.31766310051411 °
δ =  23.437796646135794 °


In [13]:
α = DECtoHMS(α)
δ = DECtoDMS(δ)

print("Pasando a HMS y DMS")
print("ρ = ", ρ, "U.A.")
print("α = ", α['hours'] , "h", α['mins'], "m", α['secs'], 's')
print("δ = ", δ['degs'], "°", δ['mins'], "'", δ['secs'], " '' " )

Pasando a HMS y DMS
ρ =  1.0161905879797808 U.A.
α =  5 h 57 m 16.239144123387348 s
δ =  23 ° 26 ' 16.067926088859394  '' 
